# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [185]:
# import libraries
import re
import pickle
import pandas as pd 
import numpy as np
from sqlalchemy import create_engine
import nltk 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split,  GridSearchCV 
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support,accuracy_score,label_ranking_average_precision_score ,confusion_matrix
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer



nltk.download(['wordnet', 'punkt', 'stopwords','omw-1.4'])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\asg19\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asg19\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asg19\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\asg19\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [188]:
# load data from database
from unicodedata import category
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('InsertTableName',engine)
df.dropna(inplace=True)
X=df['message']
Y = df[df.columns[4:]]
categories=Y.columns

### 2. Write a tokenization function to process your text data

In [189]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [190]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=200)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [191]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
model=pipeline.fit(X_train,y_train)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [1]:
def display_results(Y_test, y_pred):
    
    result=precision_recall_fscore_support(Y_test, y_pred)
    for i, col in enumerate(Y_test.columns.values):
        accu=accuracy_score(Y_test.loc[:,col],y_pred[:,i])
        score = ('{}\n Accuracy:  {:.4f}   % Precision: {:.4f}   % Recall {:.4f} '.format(
                  col,accu,result[0][i],result[1][i]))
        print(score)
    avgerage_precision = label_ranking_average_precision_score(Y_test, y_pred)
    average_score= ('average precision: {}'.format(avgerage_precision))
    print(average_score)
    

In [193]:
y_pred=pipeline.predict(X_test)


In [2]:
display_results(y_test, y_pred)

NameError: name 'y_test' is not defined

### 6. Improve your model
Use grid search to find better parameters. 

In [195]:
parameters = {
    'clf__estimator__min_samples_leaf': [1,10],
    'clf__estimator__max_features': ['auto','log2'],
}


cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1)


In [196]:
cv

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000237E4825940>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=200)))]),
             n_jobs=-1,
             param_grid={'clf__estimator__max_features': ['auto', 'log2'],
                         'clf__estimator__min_samples_leaf': [1, 10]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [197]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test) 

c:\Users\asg19\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\asg19\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\asg19\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter 

In [198]:
display_results(y_test, y_pred)

c:\Users\asg19\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\asg19\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


related
 Accuracy:  0.7956   % Precision: 0.8116   % Recall 0.9072 
request
 Accuracy:  0.8394   % Precision: 0.8338   % Recall 0.7025 
offer
 Accuracy:  0.9992   % Precision: 0.0000   % Recall 0.0000 
aid_related
 Accuracy:  0.8259   % Precision: 0.8454   % Recall 0.6984 
medical_help
 Accuracy:  0.9327   % Precision: 0.3750   % Recall 0.0180 
medical_products
 Accuracy:  0.9618   % Precision: 1.0000   % Recall 0.0400 
search_and_rescue
 Accuracy:  0.9745   % Precision: 0.0000   % Recall 0.0000 
security
 Accuracy:  0.9892   % Precision: 0.0000   % Recall 0.0000 
military
 Accuracy:  0.9936   % Precision: 0.0000   % Recall 0.0000 
child_alone
 Accuracy:  1.0000   % Precision: 0.0000   % Recall 0.0000 
water
 Accuracy:  0.9554   % Precision: 0.9545   % Recall 0.4953 
food
 Accuracy:  0.9363   % Precision: 0.9333   % Recall 0.6247 
shelter
 Accuracy:  0.9263   % Precision: 0.9135   % Recall 0.3506 
clothing
 Accuracy:  0.9920   % Precision: 0.0000   % Recall 0.0000 
money
 Accuracy:  0.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [199]:
with open('classifer.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.